# Landsat Collections

In [1]:
from geetools import collection, tools, ui

In [2]:
import ee

## Default values

In [3]:
collection.landsat.DEFAULTS

{1: {'process': 'RAW', 'sensor': 'MSS'},
 2: {'process': 'RAW', 'sensor': 'MSS'},
 3: {'process': 'RAW', 'sensor': 'MSS'},
 4: {'process': 'SR', 'sensor': 'TM'},
 5: {'process': 'SR', 'sensor': 'TM'},
 7: {'process': 'SR', 'sensor': 'ETM'},
 8: {'process': 'SR', 'sensor': 'OLI'}}

This means that the default collection for Landsat 1 is RAW and MSS, for Landsat 5 is SR and TM

# HELPERS

In [4]:
def print_values(image):
    print(tools.image.get_value(image, image.geometry().centroid(), 30, side='client'))

## Landsat 8 SR

In [5]:
l8sr = collection.Landsat(8, 'SR')

## Information about the collection

In [6]:
collection.info(l8sr)

{'algorithms': {'brdf': <bound method Landsat.brdf of <geetools.collection.landsat.Landsat object at 0x7f0e7a096d30>>,
  'harmonize': <bound method Landsat.harmonize of <geetools.collection.landsat.Landsat object at 0x7f0e7a096d30>>,
  'rescale': <bound method Landsat.rescale_all of <geetools.collection.landsat.Landsat object at 0x7f0e7a096d30>>},
 'band_names': ['aerosol',
  'blue',
  'green',
  'red',
  'nir',
  'swir',
  'swir2',
  'thermal',
  'thermal2',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'bands': ['B1',
  'B2',
  'B3',
  'B4',
  'B5',
  'B6',
  'B7',
  'B10',
  'B11',
  'sr_aerosol',
  'pixel_qa',
  'radsat_qa'],
 'bit_bands': ['sr_aerosol', 'pixel_qa', 'radsat_qa'],
 'classification_bands': [],
 'cloud_cover': 'CLOUD_COVER',
 'ee_collection': <ee.imagecollection.ImageCollection at 0x7f0e7a096eb8>,
 'end_date': '2019-04-09',
 'id': 'LANDSAT/LC08/C01/T1_SR',
 'indices': {'evi': <function geetools.collection.Collection.evi(self, name='evi', renamed=False)>,
  'nbr': <fu

## get a band by id

In [7]:
l8sr.get_band('B1').name

'aerosol'

In [8]:
l8sr.get_band('red', 'name').id

'B4'

## Get one image out of the collection

In [9]:
l8sr_i = ee.Image(l8sr.collection.first())

In [10]:
print_values(l8sr_i)

{'pixel_qa': 480, 'B10': 2639, 'B3': 8054, 'sr_aerosol': 8, 'radsat_qa': 0, 'B2': 8329, 'B5': 7478, 'B1': 8394, 'B6': 3754, 'B4': 8031, 'B7': 3120, 'B11': 2642}


# Rename bands

In [11]:
l8sr_i.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [12]:
l8sr_renamed = l8sr.rename(l8sr_i, 'all')

In [13]:
l8sr_renamed.bandNames().getInfo()

['aerosol',
 'blue',
 'green',
 'red',
 'nir',
 'swir',
 'swir2',
 'thermal',
 'thermal2',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

## Get Common Bands

In [14]:
s2toa = collection.Sentinel2TOA

In [15]:
for band in s2toa.bands:
    print(band.id, band.name)

B1 aerosol
B2 blue
B3 green
B4 red
B5 red_edge_1
B6 red_edge_2
B7 red_edge_3
B8 nir
B8A red_edge_4
B9 water_vapor
B10 cirrus
B11 swir
B12 swir2
QA10 qa10
QA20 qa20
QA60 qa60


In [16]:
l8toa = collection.Landsat8TOA

In [17]:
l8toa_i = ee.Image(l8toa.collection.first())

In [18]:
for band in l8toa.bands:
    print(band.id, band.name)

B1 aerosol
B2 blue
B3 green
B4 red
B5 nir
B6 swir
B7 swir2
B8 pan
B9 cirrus
B10 thermal
B11 thermal2
BQA bqa


In [19]:
collection.get_common_bands(l8sr, s2toa, reference='all', match='name')

['red', 'swir', 'nir', 'aerosol', 'blue', 'swir2', 'green']

## Re-scale to match Landsat 8 TOA collection

For rescaling uses `min` and `max` information for the band that match in each collection

In [20]:
# SR
l8sr.ranges('all', False)

{'B1': {'max': 10000, 'min': 0},
 'B10': {'max': 10000, 'min': 0},
 'B11': {'max': 10000, 'min': 0},
 'B2': {'max': 10000, 'min': 0},
 'B3': {'max': 10000, 'min': 0},
 'B4': {'max': 10000, 'min': 0},
 'B5': {'max': 10000, 'min': 0},
 'B6': {'max': 10000, 'min': 0},
 'B7': {'max': 10000, 'min': 0},
 'pixel_qa': {'max': None, 'min': None},
 'radsat_qa': {'max': None, 'min': None},
 'sr_aerosol': {'max': None, 'min': None}}

In [21]:
# TOA
l8toa.ranges()

{'B1': {'max': 1, 'min': 0},
 'B10': {'max': 1000, 'min': 0},
 'B11': {'max': 1000, 'min': 0},
 'B2': {'max': 1, 'min': 0},
 'B3': {'max': 1, 'min': 0},
 'B4': {'max': 1, 'min': 0},
 'B5': {'max': 1, 'min': 0},
 'B6': {'max': 1, 'min': 0},
 'B7': {'max': 1, 'min': 0},
 'B8': {'max': 1, 'min': 0},
 'B9': {'max': 1, 'min': 0},
 'BQA': {'max': None, 'min': None}}

In [22]:
for band in l8sr.bands:
    print(band.name, band.id, band.precision)

aerosol B1 int16
blue B2 int16
green B3 int16
red B4 int16
nir B5 int16
swir B6 int16
swir2 B7 int16
thermal B10 int16
thermal2 B11 int16
sr_aerosol sr_aerosol uint8
pixel_qa pixel_qa uint16
radsat_qa radsat_qa uint16


In [23]:
# l8_i_scaled_toa = l8sr.rescale_all(l8sr_i, 8, 'TOA', drop=True)
l8_toa_scaled_sr = collection.rescale(l8toa_i, l8toa, l8sr, drop=False)
print_values(l8_toa_scaled_sr)

{'B8': 0.7274518013000488, 'B10': 2638.9968872070312, 'B3': 7182.049751281738, 'B9': 0.003796438919380307, 'B2': 7998.473644256592, 'B5': 7538.013458251953, 'B1': 8138.679265975952, 'B6': 3911.091685295105, 'B4': 7489.148378372192, 'B7': 3132.632076740265, 'BQA': 2800, 'B11': 2642.7655029296875}


In [24]:
l8_i_scaled_toa = collection.rescale(l8toa_i, l8toa, l8sr, drop=True)
print_values(l8_i_scaled_toa)

{'B1': 8138.679265975952, 'B6': 3911.091685295105, 'B10': 2638.9968872070312, 'B3': 7182.049751281738, 'B4': 7489.148378372192, 'B7': 3132.632076740265, 'B5': 7538.013458251953, 'B2': 7998.473644256592, 'B11': 2642.7655029296875}


In [25]:
l8_renamed_scaled_toa = collection.rescale(l8sr_renamed, l8sr, l8toa, renamed=True, drop=True)
print_values(l8_renamed_scaled_toa)

{'red': 0.8031, 'swir': 0.3754, 'nir': 0.7478, 'swir2': 0.312, 'aerosol': 0.8394, 'thermal2': 264.2, 'green': 0.8054, 'blue': 0.8329, 'thermal': 263.90000000000003}


In [26]:
l8_renamed_scaled_toa = collection.rescale(l8sr_renamed, l8sr, l8toa, renamed=True, drop=False)
print_values(l8_renamed_scaled_toa)

{'red': 0.8031, 'swir': 0.3754, 'nir': 0.7478, 'sr_aerosol': 8, 'aerosol': 0.8394, 'blue': 0.8329, 'radsat_qa': 0, 'pixel_qa': 480, 'swir2': 0.312, 'thermal2': 264.2, 'green': 0.8054, 'thermal': 263.90000000000003}


## Re-scale a collection

In [27]:
col = l8sr.collection.limit(5)

In [28]:
col_renamed = col.map(lambda img: l8sr.rename(img))

In [29]:
col_reescaled = col_renamed.map(lambda img: collection.rescale(img, l8sr, l8toa, drop=False, renamed=True))

In [30]:
tools.imagecollection.get_values(col_reescaled, ee.Image(col.first()).geometry().centroid(), 30, side='client')

{'LC08_001004_20140524': {'aerosol': 0.8394,
  'blue': 0.8329,
  'green': 0.8054,
  'nir': 0.7478,
  'pixel_qa': 480.0,
  'radsat_qa': 0.0,
  'red': 0.8031,
  'sr_aerosol': 8.0,
  'swir': 0.3754,
  'swir2': 0.312,
  'thermal': 263.90000000000003,
  'thermal2': 264.2},
 'LC08_001004_20140609': {'aerosol': 0.5276,
  'blue': 0.5043,
  'green': 0.4709,
  'nir': 0.4036,
  'pixel_qa': 480.0,
  'radsat_qa': 0.0,
  'red': 0.4519,
  'sr_aerosol': 8.0,
  'swir': 0.088,
  'swir2': 0.0902,
  'thermal': 260.79999999999995,
  'thermal2': 260.90000000000003},
 'LC08_001004_20140625': {'aerosol': 0.4231,
  'blue': 0.4203,
  'green': 0.4111,
  'nir': 0.4199,
  'pixel_qa': 480.0,
  'radsat_qa': 0.0,
  'red': 0.4133,
  'sr_aerosol': 8.0,
  'swir': 0.3932,
  'swir2': 0.3506,
  'thermal': 268.2,
  'thermal2': 268.8},
 'LC08_001004_20140711': {'aerosol': 0.8471,
  'blue': 0.8566,
  'green': 0.7929,
  'nir': 0.5672,
  'pixel_qa': 336.0,
  'radsat_qa': 0.0,
  'red': 0.7489,
  'sr_aerosol': 196.0,
  'swir': 0.

# From Id

In [31]:
l5toa = collection.Landsat.fromId('LANDSAT/LT05/C01/T1_TOA')

In [32]:
collection.info(l5toa)

{'algorithms': {'brdf': <bound method Landsat.brdf of <geetools.collection.landsat.Landsat object at 0x7f0eb404e400>>,
  'harmonize': <bound method Landsat.harmonize of <geetools.collection.landsat.Landsat object at 0x7f0eb404e400>>,
  'rescale': <bound method Landsat.rescale_all of <geetools.collection.landsat.Landsat object at 0x7f0eb404e400>>},
 'band_names': ['blue',
  'green',
  'red',
  'nir',
  'swir',
  'thermal',
  'swir2',
  'bqa'],
 'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'BQA'],
 'bit_bands': ['BQA'],
 'classification_bands': [],
 'cloud_cover': 'CLOUD_COVER',
 'ee_collection': <ee.imagecollection.ImageCollection at 0x7f0e79fc5dd8>,
 'end_date': '2012-05-05',
 'id': 'LANDSAT/LT05/C01/T1_TOA',
 'indices': {'evi': <function geetools.collection.Collection.evi(self, name='evi', renamed=False)>,
  'nbr': <function geetools.collection.Collection.nbr(self, name='nbr', renamed=False)>,
  'ndvi': <function geetools.collection.Collection.ndvi(self, name='ndvi', renamed=Fa

In [33]:
l5toa.get_band('B3').name

'red'

## Visualize

In [34]:
Map = ui.Map()

In [35]:
Map.show()

Map(basemap={'max_zoom': 19, 'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map …

In [37]:
rect = ee.Geometry.Rectangle([-71.57, -40.81, -71.43, -40.72])

In [38]:
Map.addLayer(rect)

In [39]:
p = ee.Geometry.Point([-71.63085937500001, -40.76448635693034])

In [40]:
l8sri = l8sr.collection.filterBounds(p).filterMetadata(l8sr.cloud_cover, 'less_than', 30).first()

In [42]:
vis = l8sr.visualization('NSR')

In [43]:
vis

{'bands': ['B5', 'B6', 'B4'],
 'max': [5000.0, 5000.0, 5000.0],
 'min': [0, 0, 0]}

In [44]:
Map.addLayer(l8sri, vis, 'L8 SR NSR')

In [45]:
Map.centerObject(l8sri)

In [52]:
s2sr.visualization('NSR')

{'bands': ['B8', 'B11', 'B4'],
 'max': [5000.0, 5000.0, 5000.0],
 'min': [0, 0, 0]}

In [53]:
Map.addLayer(s2sri, visS2, 'Sentinel 2 SCL')

In [54]:
Map.addLayer(i2, {'bands':['B8', 'B11', 'B4'], 'max':4000, 'min':0}, 'Sentinel 2 NSR')

In [55]:
s2sr.SCL_data

{1: 'saturated',
 2: 'dark',
 3: 'shadow',
 4: 'vegetation',
 5: 'bare_soil',
 6: 'water',
 7: 'cloud_low',
 8: 'cloud_medium',
 9: 'cloud_high',
 10: 'cirrus',
 11: 'snow'}

In [56]:
proxy = l8toa.proxy_image()

In [57]:
ui.eprint(proxy, notebook=True)